# Sub Net

In [1]:
import socket, os
import numpy as np
from tensorflow.keras import layers as KL
from tqdm import tqdm
import glob, copy
import scipy
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import backend as K
import sys
# from freesurfer import deeplearn as fsd
import freesurfer as fs
from freesurfer.lookups import nonlateral_aseg_recoder

import neurite as ne
import neurite_sandbox as nes
import voxelmorph as vxm
import voxelmorph_sandbox as vxms
from pathlib import Path
from tensorflow.keras.utils import to_categorical
import surfa as sf
from utils import *

import layer_dict as ld
import pdb as gdb

from tensorflow.keras.optimizers import Adam


log_dir = "logs"
models_dir = "models"

if not os.path.exists(log_dir):
    os.makedirs(log_dir)

if not os.path.exists(models_dir):
    os.makedirs(models_dir)
    
weights_saver = PeriodicWeightsSaver(filepath=models_dir, save_freq=10)  # Save weights every 100 epochs

TB_callback = CustomTensorBoard(
    base_log_dir=log_dir,
    histogram_freq=100,
    write_graph=True,
    write_images=False,
    write_steps_per_second=False,
    update_freq='epoch',
    profile_batch=0,
    embeddings_freq=0,
    embeddings_metadata=None
)
            
vscale = 2
vscale = 1

dofit = True

which_loss = 'both'
which_loss = 'mse'
which_loss = 'cce'
which_loss = 'dice'

same_contrast=False
same_contrast=True

oshapes = False
oshapes = True

fit_lin = True
fit_lin = False

save_model = False
save_model = True

doaff = False
doaff = True

test_adni = False


model_dir = 'models'
gpuid = -1
host = socket.gethostname()
from neurite_sandbox.tf.utils.utils import plot_fit_callback as pfc


print(f'host name {socket.gethostname()}')
print("visible devices:", os.environ["CUDA_VISIBLE_DEVICES"])
# ngpus = 1 if os.getenv('NGPUS') is None else int(os.getenv('NGPUS'))
ngpus =len(os.environ["CUDA_VISIBLE_DEVICES"])
print(f'using {ngpus} gpus')
if ngpus > 1:
    model_device = '/gpu:0'
    synth_device = '/gpu:1'
    synth_gpu = 1
    dev_str = "0, 1"
else:
    model_device = '/gpu:0'
    synth_device = model_device
    synth_gpu = 0
    dev_str = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = dev_str
print(f'model_device {model_device}, synth_device {synth_device}, dev_str {dev_str}')
print(f'physical GPU # is {os.getenv("SLURM_STEP_GPUS")}')
ret = ne.utils.setup_device(dev_str)


2024-02-06 14:49:08.530454: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-06 14:49:08.576012: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Issue loading cv2


host name rtx-08.nmr.mgh.harvard.edu
visible devices: 0,1
using 3 gpus
model_device /gpu:0, synth_device /gpu:1, dev_str 0, 1
physical GPU # is None


In [1]:
# print(model_device,synth_device,ngpus,os.getenv('NGPUS'),os.environ["CUDA_VISIBLE_DEVICES"])
!pip install pytest-shutil

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
import linecache
import os, psutil
from itertools import islice

ntest = 5
num_subjects=10
# policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')

# policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
# tf.keras.mixed_precision.experimental.set_policy(policy)
# policy = tf.keras.mixed_precision.Policy('float32')
# tf.keras.mixed_precision.set_global_policy(policy)

print(f'dofit {dofit}, doaff {doaff}, fit_lin {fit_lin}, oshapes {oshapes}, save_model {save_model}')


batch_size = 1

adir = '/autofs/cluster/freesurfer/subjects/atlases/aseg_atlas'
mname = 'seg_edited.mgz'
vname = 'norm.mgz'
sfile = os.path.join(adir, 'scripts', 'subjects.txt')

# man_subjects = [linecache.getline(sfile, i).strip() for i in range(1, num_subjects + 1)]
man_subjects = [line.strip() for line in islice(open(sfile), num_subjects)]


crop = -1 if dofit else ntest

# odir = '/autofs/vast/braindev/braindev/OASIS/OASIS1/synth-high-res/recon_subject'
# subjects = [f for f in Path(odir).iterdir() if 'OASIS_OAS1_0' in str(f)]
# seg_files = [f/f'mri/aparc+aseg.mgz' for f in tqdm(subjects)]
# seg_files = [f/f'mri/aseg.mgz' for f in tqdm(subjects)]
odir = '/autofs/vast/braindev/braindev/OASIS/OASIS1/synth-high-res/recon_subject'
# subjects = [f for f in Path(odir).iterdir() if 'OASIS_OAS1_0' in str(f)]
subjects = [f for f in islice(Path(odir).iterdir(), num_subjects) if 'OASIS_OAS1_0' in str(f)]

seg_files = [f / 'mri/aparc+aseg.mgz' for f in tqdm(subjects[0:crop])]

print(len(seg_files),len(subjects))
if dofit:
    print(f'TRAINING model with loss {which_loss}')
else:
    print(f'loading model trained with {which_loss} loss')

target_shape = (192,)*3
inshape = target_shape

def minmax_norm(mri, axis=None):
    """
    Min-max normalize an mri struct using a safe division.

    Arguments:
        x: np.array to be normalized
        axis: Dimensions to reduce during normalization. If None, all axes will be considered,
            treating the input as a single image. To normalize batches or features independently,
            exclude the respective dimensions.

    Returns:
        Normalized tensor.
    """
    x = mri.data
    x_min = x.min()
    x_max = x.max()
    mri.data = (x - x_min) / (x_max - x_min) 
    return mri




lut = fs.lookups.default()
# lut = fs.lookups.LookupTable.read(os.path.join(odir, 'seg35_labels.txt'))
# lut = fs.lookups.LookupTable.read(os.path.join(odir, f'seg{nb_labels}_labels.txt'))

# csf = lut.search('CSF')[0]
lesion_label_orig = lut.search('Left-Lesion')
if len(lesion_label_orig) > 0:
    lesion_label_orig = lesion_label_orig[0]
else:   # not in the lut - add a new one
    lesion_label_orig = 77
    lut.add(lesion_label_orig, 'Lesion', color=[240,240,240])

if 'inited' not in locals() and 'inited' not in globals():
    inited = False

# print("lut:",lut)

if not inited:
    # mri_segs_orig = [fs.Volume.read(str(fname)) for fname in tqdm(seg_files[:crop])]
    mri_segs_orig = [sf.load_volume(str(fname)) for fname in tqdm(seg_files[:crop])]

    # mri_segs = [sf.load_volume(str(fname)) for fname in tqdm(seg_files[:crop])]
    if vscale > 1:
        print(f'downsampling by {vscale}')
        mri_segs = [mri.reslice(vscale) for mri in tqdm(mri_segs_orig)]
    else:
        print(f'cropping to {target_shape}')
        mri_segs = [mri.fit_to_shape(target_shape, center='bbox') for mri in tqdm(mri_segs_orig)]
        # mri_segs = [mri.reshape(target_shape, center='bbox') for mri in tqdm(mri_segs)]

    np_segs_orig = [mri.data for mri in mri_segs]
    print(f'finding unique labels in {len(mri_segs_orig)} datasets...')
    labels_orig = np.unique(np.array(np_segs_orig))
    print("labels orig:",labels_orig)
    # mapping = fs.lookups.tissue_type_recoder_no_skull(include_lesions=use_lesions)
    mapping = fs.lookups.nonlateral_aseg_recoder()
    target_lut = mapping.target_lut
    print("target_lut: ",target_lut)
    lut_name = 'nonlat.txt'
    mri_segs_recoded = [fs.label.recode(mri, mapping) for mri in mri_segs]  
    print("mri_segs_recoded: ", mri_segs_recoded[0].shape,np.max(mri_segs_recoded[0]))
    lesion_label = target_lut.search('Left-Lesion')[0]
    print("lesion_label:",lesion_label)
    np_segs = [vol.data for vol in mri_segs_recoded]
    print("vol:", mri_segs_recoded[0].shape)
    labels_in = np.unique(np.array(np_segs)).astype(int)
    print("labels in: ", labels_in)
    if lesion_label not in labels_in:
        l = list(labels_in)
        l.append(lesion_label)
        labels_in = np.array(l)
    nlabels_small = len(labels_in)
    label_map = {}
    keys = mapping.mapping.keys()
    # print("keys:",keys)
    lab_to_ind = np.zeros((labels_orig.max()+1,), dtype=np.uint8)
    for label in labels_orig:
        if label not in keys:
            output_label = 0
        else:
            output_label = mapping.mapping[label]
        label_map[label] = output_label
        lab_to_ind[label] = output_label


    mri_man_segs = []  # manual segs
    mri_norms = []  # mri vols
    mri_norms_orig = []
    mri_man_segs_orig = []

    for s in tqdm(man_subjects):
        # mri_seg_orig = fs.Volume.read(os.path.join(adir, s, 'mri', mname))
        print("mname:",os.path.join(adir, s, 'mri', mname))
        mri_seg_orig = sf.load_volume(os.path.join(adir, s, 'mri', mname))

    
        mri_man_segs_orig.append(mri_seg_orig)
        # mri_seg = mri_seg_orig.fit_to_shape(target_shape, center='bbox')
        mri_seg = mri_seg_orig.reshape(target_shape)
        mri_man_segs.append(mri_seg)
        # mri_norm_orig = fs.Volume.read(os.path.join(adir, s, 'mri', vname))
        mri_norm_orig = sf.load_volume(os.path.join(adir, s, 'mri', vname))

        mri_norm = mri_norm_orig.resample_like(mri_seg)
        mri_norms.append(mri_norm)
        mri_norms_orig.append(mri_norm_orig)

    mri_man_segs_recoded = [fs.label.recode(mri, mapping) for mri in tqdm(mri_man_segs)]

    # mri_seg_atlas = fs.Volume.read("aseg_atlas.mgz")
    mri_seg_atlas = sf.load_volume("aseg_atlas.mgz").reshape(target_shape)
    print("mri_seg_atlas:",mri_seg_atlas.shape)
    hard_seg = np.argmax(mri_seg_atlas.data, axis=-1)
    mri_hard_seg = mri_seg_atlas.copy()
    mri_hard_seg.data = hard_seg
    mri_hard_seg_cropped = mri_hard_seg.reshape(target_shape)
    print(mri_hard_seg_cropped.shape)
    mri_norm_atlas = sf.load_volume("norm_atlas.mgz").resample_like(mri_hard_seg)
    print("mri_norm_atlas:",mri_norm_atlas.shape)
    # mri_hard_seg = mri_seg_atlas.copy(hard_seg).fit_to_shape(target_shape, center='bbox')
    # mri_norm_atlas = fs.Volume.read("norm_atlas.mgz").resample_like(mri_hard_seg)

    mri_seg_atlas = mri_seg_atlas.resample_like(mri_hard_seg)
    norm_atlas = (mri_norm_atlas.data / mri_norm_atlas.data.max())[np.newaxis, ..., np.newaxis]
    print("norm atlas shape: ", norm_atlas.shape,mri_hard_seg.shape,mri_seg_atlas.shape)
    inited = True


dofit True, doaff True, fit_lin False, oshapes True, save_model True


100%|██████████| 9/9 [00:00<00:00, 67893.41it/s]


9 10
TRAINING model with loss dice


100%|██████████| 8/8 [00:01<00:00,  5.37it/s]


cropping to (192, 192, 192)


...

mname: /autofs/cluster/freesurfer/subjects/atlases/aseg_atlas/990114_vc722/mri/seg_edited.mgz


 90%|█████████ | 9/10 [00:01<00:00,  4.44it/s]

mname: /autofs/cluster/freesurfer/subjects/atlases/aseg_atlas/990114_vc723/mri/seg_edited.mgz


100%|██████████| 10/10 [00:10<00:00,  1.03s/it]


mri_seg_atlas: (192, 192, 192, 17)
(192, 192, 192)
mri_norm_atlas: (192, 192, 192)
norm atlas shape:  (1, 192, 192, 192, 1) (192, 192, 192) (192, 192, 192, 17)


In [4]:
print(np.sum(mri_man_segs_recoded[0]))

2590041


# Section 2

In [5]:
warp_max=2.5
warp_max=2.1
warp_max=2   
warp_min=.5
warp_blur_min=np.array([2, 4, 8])
warp_blur_max=warp_blur_min*2
bias_blur_min=np.array([2, 4, 8])
bias_blur_max=bias_blur_min*2

fscale = 1
fscale = 1.75
fscale = 1.5   # matches P32 N 16
fscale = 1.1
nfeats = 64
nfeats = 42
nsmall = 0
nb_levels = int(np.log2(inshape[0]))-(1+nsmall)   # 4,4,4 is lowest level
nb_conv_per_level = 2
unet_scale = 1
nfeats_small = int(nfeats // 3)
unet_nf = []


print(f'using warp max = {warp_max} and nlabels {nlabels_small}, fscale {fscale}')
inshape=np_segs[0].shape
gen_args = dict(
    warp_min=warp_min,
    warp_max=warp_max,
    blur_max=2,  # was .5, then 1
    bias_max=.25,  # was 2 then .5
    bias_blur_min=bias_blur_min,
    bias_blur_max=bias_blur_max,
    gamma=0,
    # warp_zero_mean=True,
    zero_background=.75,
    noise_max=.2,   
    noise_min=.1
)

def synth_gen(label_vols, gen_model, lab_to_ind, labels_in, batch_size=8, use_rand=True, gpuid=1, 
              seg_resize=1, num_outside_shapes_to_add=8, use_log=False, debug=False, 
              add_outside=True):

    inshape = label_vols[0].shape
    nlabels = gen_model.outputs[-1].get_shape().as_list()[-1]  # number of compressed labels

    if add_outside:
        l2l = nes.models.labels_to_labels(
            labels_in,
            shapes_num=num_outside_shapes_to_add,
            in_shape=label_vols[0].shape,
            shapes_add=True
        )
        li = np.concatenate([labels_in, np.arange(labels_in.max()+1, 
                                                  labels_in.max()+1+num_outside_shapes_to_add)])
        l2i = nes.models.labels_to_image(
            labels_in=li,
            labels_out=None,
            in_shape=label_vols[0].shape,
            zero_background=.5,  #  was 1
            noise_max=.2,
            noise_min=.1,
            warp_max=0
        )

    # outputs [6] and [7] are the t2 labels without (6) and with (7) atrophy
    batch_input_labels = np.zeros((batch_size, *inshape, 1))
    label_shape = tuple(np.array(inshape) // seg_resize)
    batch_onehots = np.zeros((batch_size, *label_shape, nlabels))
    batch_images = np.zeros((batch_size, *inshape, 1))

    if debug:
        batch_orig_images1 = np.zeros((batch_size, *inshape, 1))
        batch_orig_images2 = np.zeros((batch_size, *inshape, 1))
        batch_orig_labels1 = np.zeros((batch_size, *label_shape, 1))
        batch_orig_labels2 = np.zeros((batch_size, *label_shape, 1))

    if gpuid >= 0:
        device = '/gpu:' + str(gpuid)
    else:
        device = '/physical_device:CPU:0'
        device = '/cpu:0'
    print(device)
    ind = -1
    while (True):
        for bind in range(batch_size):
            if use_rand:
                ind = np.random.randint(0, len(label_vols))
            else:
                ind = np.mod(ind+1, len(label_vols))
            batch_input_labels[bind,...] = label_vols[ind].data[...,np.newaxis]

        with tf.device(device):
            pred = gen_model.predict_on_batch(batch_input_labels)

        for bind in range(batch_size):
            im = pred[0][bind,...]
            onehot = pred[1][bind,...]
            # if add_outside:
                # im = nes.utils.augment.add_outside_shapes(im[..., 0], np.argmax(onehot, axis=-1), labels_in, l2l=l2l, l2i=l2i)[..., np.newaxis]

            if use_log:
                onehot[onehot == 0] = -10
                onehot[onehot == 1] = 10

            batch_images[bind, ...] = im
            batch_onehots[bind, ...] = onehot

        inputs = [batch_images]
        outputs = [batch_onehots]

        yield inputs, outputs
                
    return 0



for level in range(nb_levels-nsmall):
    filters_in_this_level = []
    for layer in range(nb_conv_per_level):
        filters_in_this_level.append(int(fscale**level*nfeats))
        
    unet_nf.append(filters_in_this_level)

print("len unet_nf",len(unet_nf), unet_nf)
unet_nf = [[nfeats_small] * nb_conv_per_level] * nsmall + unet_nf



using warp max = 2 and nlabels 17, fscale 1.1
len unet_nf 6 [[42, 42], [46, 46], [50, 50], [55, 55], [61, 61], [67, 67]]


In [6]:
import generators as gens

tf.compat.v1.enable_eager_execution()

lr = 1e-5
lr = 1e-4
lr_lin = 1e-4
thresh = -.2*2 
cooldown = 25
patience = 600

name = f'aseg.outside.unet_nf.{nfeats}.{nfeats_small}.{nsmall}.levels.{nb_levels}.warp_max.{warp_max}.oshapes.{oshapes}.fscale.{fscale}'

label_weights = np.ones((1,nlabels_small,))
# label_weights[0,-1] = .01  # downweight lesion class
lfunc = ne.losses.Dice(nb_labels=nlabels_small, weights=None, check_input_limits=False).mean_loss


mc_cb_lin = tf.keras.callbacks.ModelCheckpoint(name+'.checkpoint.lin.tf', save_best_only=False)
write_cb_lin = nes.callbacks.WriteHist(name+'.lin.txt')

unet_device = model_device if (fit_lin or dofit) else synth_device
with tf.device(unet_device): 
    model_lin = ne.models.unet(unet_nf, inshape+(1,), None, 3, nlabels_small, feat_mult=None, final_pred_activation='linear')
    print(f'unet model created with nf {unet_nf}')
    softmax_out = KL.Softmax(name='seg')(model_lin.outputs[0])
    model = tf.keras.Model(model_lin.inputs, [softmax_out])
    model.summary()

vxm_model = gens.read_vxm_model(inshape)
vxm_smooth_wt = np.zeros((1, 1))
vxm_smooth_wt[0,0] = .3   # warp regularization hyper parameter

gen_model = gens.create_gen_model(np_segs, oshapes, synth_device, nlabels_small, labels_in, inshape, warp_max)

if fit_lin and dofit:
    thresh_lin = 5
    thresh = -.5
    print(f'thresh {thresh}, thresh_lin {thresh_lin}')
    lr_cb_lin = nes.tf.callbacks.ReduceLRWithModelCheckpointAndRecovery(name+'.lin.h5', monitor='loss',
                                                                        verbose=1, cooldown=cooldown, 
                                                                        recovery_decrease_factor=.8,
                                                                        factor=.8, patience=patience, 
                                                                        thresh_increase_factor=2,
                                                                        thresh=thresh_lin,
                                                                        save_weights_only=True, 
                                                                        burn_in=10,
                                                                        min_lr=1e-7,
                                                                        warm_restart_epoch=None,
                                                                        nloss=5,
                                                                        warm_restart_lr=None)

    callbacks_lin = [lr_cb_lin, write_cb_lin, ne.callbacks.LRLog(), mc_cb_lin]
    callbacks_lin = [lr_cb_lin, write_cb_lin, ne.callbacks.LRLog()]
    lfunc_lin = ne.losses.MeanSquaredErrorProb().loss
    with tf.device(synth_device):
        gen_lin = synth_gen(mri_segs_recoded, gen_model, None, labels_in, batch_size=batch_size, 
                            use_rand=True, gpuid=synth_gpu,use_log=True, add_outside=oshapes)
        gen = synth_gen(mri_segs_recoded, gen_model, None, labels_in, batch_size=batch_size, 
                        use_rand=True, gpuid=synth_gpu, debug=False, add_outside=oshapes)
        vgen = gens.real_gen(mri_man_segs_recoded, mri_norms, vxm_model, norm_atlas, 
                             None, labels_in, 
                             batch_size=batch_size, use_rand=True, 
                             gpuid=synth_gpu, debug=False, add_outside=oshapes)

    loss_funcs_lin = [lfunc_lin]
    print(f'fitting linear model and saving hist to {write_cb_lin.fname} and model to {lr_cb_lin.fname}')
    with tf.device(model_device):
        nes.utils.check_and_compile(model_lin, gen_lin, optimizer= Adam(learning_rate=lr_lin), 
                                    loss=loss_funcs_lin, check_layers=False, run_eagerly=True)
        fhist_lin = model_lin.fit(gen_lin, epochs=int(50), steps_per_epoch=50, initial_epoch=0, callbacks=callbacks_lin)
        model_lin.save_weights(lr_cb_lin.fname)
else:
    fname = name + '.lin.h5'
    if os.path.exists(fname) and dofit:
        print(f'loading linear weights from {fname}')
        model_lin.load_weights(fname)


2024-02-06 14:50:16.379287: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46871 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:3e:00.0, compute capability: 8.6
2024-02-06 14:50:16.380159: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 47377 MB memory:  -> device: 1, name: Quadro RTX 8000, pci bus id: 0000:3f:00.0, compute capability: 7.5


unet model created with nf [[42, 42], [46, 46], [50, 50], [55, 55], [61, 61], [67, 67]]
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 unet_input (InputLayer)     [(None, 192, 192, 192, 1)]   0         []                            
                                                                                                  
 unet_conv_downarm_0_0 (Con  (None, 192, 192, 192, 42)    1176      ['unet_input[0][0]']          
 v3D)                                                                                             
                                                                                                  
 unet_conv_downarm_0_1 (Con  (None, 192, 192, 192, 42)    47670     ['unet_conv_downarm_0_0[0][0]'
 v3D)                                                               ]                             
      

(None, 192, 192, 192, 1)
(None, 192, 192, 192, 1)


# Section 3

In [ ]:
patch_size = 16
initial_epoch = 0
ntest = 5
num_epochs = 2
with tf.device(synth_device):
    val_size = 20
    gen = gens.synth_gen(mri_segs_recoded, gen_model, vxm_model, norm_atlas, 
                         None, labels_in, batch_size=batch_size, 
                         use_rand=True, gpuid=synth_gpu, debug=False, add_outside=oshapes,
                         zero_background=.1)
    vgen = gens.real_gen(mri_man_segs_recoded, mri_norms, vxm_model, norm_atlas, 
                         None, labels_in, 
                         batch_size=batch_size, use_rand=True, 
                         gpuid=synth_gpu, debug=False, add_outside=oshapes)
    if 0:
        vgen = gens.synth_gen(mri_segs_recoded[len(mri_segs_recoded)-val_size:], gen_model, None, labels_in, 
                              batch_size=batch_size, use_rand=True, 
                              gpuid=synth_gpu, debug=False, add_outside=oshapes)



write_cb = nes.callbacks.WriteHist(name+'.txt', mode='w' if initial_epoch == 0 else 'a')
if initial_epoch > 0:
    initial_epoch = write_cb.start_epoch
    print(f'loading old model and restarting from epoch {initial_epoch}')
    model = tf.keras.models.load_model(name+'.checkpoint.h5')#, custom_objects=ld.layer_dict)

mc_cb = tf.keras.callbacks.ModelCheckpoint(name+'.checkpoint.h5', save_best_only=False)
lr_cb = nes.tf.callbacks.ReduceLRWithModelCheckpointAndRecovery(name+'.h5', monitor='loss',
                                                                    verbose=2, cooldown=cooldown, 
                                                                    recovery_decrease_factor=1,
                                                                    factor=.8, patience=patience, 
                                                                    thresh_increase_factor=1.2,
                                                                    thresh=thresh,
                                                                    save_weights_only=True, 
                                                                    burn_in=50,
                                                                    min_lr=1e-7,
                                                                    restart=initial_epoch > 0,
                                                                    nloss=5)

# callbacks = [lr_cb, write_cb, ne.callbacks.LRLog(), mc_cb]
callbacks = [TB_callback,weights_saver]#, ne.callbacks.LRLog(), mc_cb]

# callbacks = [lr_cb, write_cb, ne.callbacks.LRLog(), mc_cb]

# callbacks = [mc_cb]

with tf.device(model_device):
    print("checking and compiling model:")
    
    nes.utils.check_and_compile(model, gen, optimizer=Adam(learning_rate=lr), 
                                loss=[lfunc], check_layers=False, run_eagerly=True)
    
    print(f"{'saving' if dofit else 'loading'} fit results to {lr_cb.fname} and hist to {write_cb.fname}")
    if dofit:
        # model.load_weights('aseg.h5')
        # fhist = model.fit(gen, epochs=int(num_epochs), steps_per_epoch=50, 
        #                   initial_epoch=initial_epoch, callbacks=callbacks, 
        #                   validation_data=vgen, validation_steps=5)
        fhist = model.fit(gen, epochs=int(num_epochs), steps_per_epoch=50, 
                  initial_epoch=initial_epoch, callbacks=callbacks, 
                  validation_data=vgen, validation_steps=5)
    else:
        
        # model.load_weights(lr_cb.fname)
        print(name + '.checkpoint.h5')
        model = tf.keras.models.load_model(name + '.checkpoint.h5', custom_objects=ld.layer_dict)
    

if save_model:
    aseg_fname = f'aseg.fscale.{fscale}.h5'
    print(f'saving model to {aseg_fname}')
    model.save(aseg_fname)


vgen = gens.real_gen(mri_man_segs_recoded, mri_norms, vxm_model, norm_atlas, 
                     None, labels_in, 
                     batch_size=1, use_rand=True, 
                     gpuid=synth_gpu, debug=False, add_outside=oshapes)
gen = gens.synth_gen(mri_segs_recoded, gen_model, vxm_model, norm_atlas, 
                     None, labels_in, batch_size=1, use_rand=True, gpuid=synth_gpu, debug=False, 
                     zero_background=.1,
                     add_outside=oshapes)

ilist = []
olist = []
plist = []
dlist = []
choroid_label = target_lut.search('Left-Choroid')[0]
mask = np.ones((nlabels_small,))
mask[lesion_label] = 0
mask[choroid_label] = 0
mask[0] = 0
lfunc_dice = ne.losses.Dice(nb_labels=nlabels_small, weights=None, check_input_limits=False).loss
# for n in tqdm(range(ntest)):
for n in tqdm(range(1)):
    inb, outb = next(vgen)
    pred = model.predict(inb)
    # d = model.evaluate(inb, outb, verbose=0)
    d = lfunc_dice(tf.convert_to_tensor(outb, tf.float32), tf.convert_to_tensor(pred, tf.float32))
    d = (d.numpy() * mask).sum() / mask.sum()
    dlist.append(d)
    print(inb[0].shape)
    ilist.append(inb[0].squeeze().copy())
    olist.append(np.argmax(outb[0].squeeze(), axis=-1).copy())
    plist.append(np.argmax(pred[0].squeeze(), axis=-1).copy())


print(f'real dice {np.array(dlist).mean()}')
print(f'{dlist}')





lr callback created with thresh -0.4
checking and compiling model:
batch inpu: (1, 192, 192, 192, 1)  batch norm atlas: (1, 192, 192, 192, 1)


saving fit results to aseg.outside.unet_nf.42.14.0.levels.6.warp_max.2.oshapes.True.fscale.1.1.h5 and hist to aseg.outside.unet_nf.42.14.0.levels.6.warp_max.2.oshapes.True.fscale.1.1.txt
batch inpu: (1, 192, 192, 192, 1)  batch norm atlas: (1, 192, 192, 192, 1)
Epoch 1/2
batch inpu: (1, 192, 192, 192, 1)  batch norm atlas: (1, 192, 192, 192, 1)
batch inpu: (1, 192, 192, 192, 1)  batch norm atlas: (1, 192, 192, 192, 1)
1/1 [==============================] - 2s 2s/step- loss: -0.25
batch inpu: (1, 192, 192, 192, 1)  batch norm atlas: (1, 192, 192, 192, 1)
1/1 [==============================] - 3s 3s/step
batch inpu: (1, 192, 192, 192, 1)  batch norm atlas: (1, 192, 192, 192, 1)
50/50 [==============================] - 244s 5s/step - loss: -0.2595 - val_loss: -0.4003
Epoch 2/2
batch inpu: (1, 192, 192, 192, 1)  batch norm atlas: (1, 192, 192, 192, 1)
50/50 [==============================] - ETA: 0s - loss: -0.4521

In [19]:
imgs = np.array(ilist)
tseg = np.array(olist)
pseg = np.array(plist)
print(imgs.shape,tseg.shape,pseg.shape)
import nibabel as nib
import numpy as np

# print(imgs[0].shape)
for i in range(1):
    nib.save(nib.Nifti1Image(imgs[i].astype(np.float32), np.eye(4), header=None), f"output/image{i}.nii.gz")
    nib.save(nib.Nifti1Image(tseg[i].astype(np.int32), np.eye(4), header=None), f"output/tseg{i}.nii.gz")
    nib.save(nib.Nifti1Image(pseg[i].astype(np.int32), np.eye(4), header=None), f"output/pseg{i}.nii.gz")

# fv = fs.Freeview(swap_batch_dim=True)
# fv.vol(imgs, name='img', opts=':locked=1:linked=1')

(1, 192, 192, 192) (1, 192, 192, 192) (1, 192, 192, 192)


In [11]:
for n in tqdm(range(1)):
    inb, outb = next(vgen)
    pred = model.predict(inb)
    # d = model.evaluate(inb, outb, verbose=0)
    d = lfunc_dice(tf.convert_to_tensor(outb, tf.float32), tf.convert_to_tensor(pred, tf.float32))
    d = (d.numpy() * mask).sum() / mask.sum()
    dlist.append(d)
    print(inb[0].shape)
    ilist.append(inb[0].squeeze().copy())
    olist.append(np.argmax(outb[0].squeeze(), axis=-1).copy())
    plist.append(np.argmax(pred[0].squeeze(), axis=-1).copy())


print(f'real dice {np.array(dlist).mean()}')
print(f'{dlist}')


  0%|          | 0/1 [00:00<?, ?it/s]

1/1 [==============================] - 134s 134s/step
(1, 192, 192, 192, 1)


100%|██████████| 1/1 [05:33<00:00, 333.72s/it]

real dice -0.8219432532787323
[-0.8219432532787323]


In [15]:
print(imgs.shape[0])

5


# Test

In [ ]:
test_mseg = True
if test_mseg:
    aseg_atlas_fname = 'aseg.fscale.1.1.h5'
    print(f'reading aseg atlas model from {aseg_atlas_fname}')
    atlas_model = tf.keras.models.load_model(aseg_atlas_fname, custom_objects=ld.layer_dict)
    mapping = fs.lookups.nonlateral_aseg_recoder()
    target_lut = mapping.target_lut
    lut_name = 'nonlat.txt'
    adir = '/autofs/cluster/freesurfer/subjects/atlases/aseg_atlas'
    mname = 'seg_edited.mgz'
    vname = 'norm.mgz'
    sfile = os.path.join(adir, 'scripts', 'subjects.txt')
    with open(sfile, 'r') as f:
        subjects = f.read().split('\n')[0:-1]

    mri_man_segs = []  # manual segs
    mri_norms = []  # mri vols
    mri_norms_orig = []
    mri_man_segs_orig = []
    for s in tqdm(subjects):
        # mri_seg_orig = fs.Volume.read(os.path.join(adir, s, 'mri', mname))
        mri_seg_orig = sf.load_volume(os.path.join(adir, s, 'mri', mname))
        mri_man_segs_orig.append(mri_seg_orig)
        # mri_seg = mri_seg_orig.fit_to_shape(target_shape, center='bbox')
        mri_seg = mri_seg_orig.reshape(target_shape)
        mri_man_segs.append(mri_seg)
        mri_norm_orig = sf.load_volume(os.path.join(adir, s, 'mri', vname))
        mri_norm = mri_norm_orig.resample_like(mri_seg)
        mri_norms.append(mri_norm)
        mri_norms_orig.append(mri_norm_orig)

    mri_man_segs_recoded = [fs.label.recode(mri, mapping) for mri in tqdm(mri_man_segs)]

    mri_seg_atlas = sf.load_volume("aseg_atlas.mgz")
    hard_seg = np.argmax(mri_seg_atlas.data, axis=-1)
    # mri_hard_seg = mri_seg_atlas.copy(hard_seg).fit_to_shape(target_shape, center='bbox')
    mri_hard_seg = sf.Volume(hard_seg).resample_like(mri_seg_atlas).reshape(target_shape)
    mri_norm_atlas = sf.load_volume("norm_atlas.mgz").resample_like(mri_hard_seg)
    mri_seg_atlas = mri_seg_atlas.resample_like(mri_hard_seg)
    norm_atlas = (mri_norm_atlas.data / mri_norm_atlas.data.max())[np.newaxis, ..., np.newaxis]

    # psize = (norm_atlas.shape[1] - mri_norms[0].shape[0]) // 2
    # pad = ((0,0), (psize,psize), (psize, psize), (psize, psize), (0, 0))
    dice_list = []
    elist = []
    elist_in_atlas = []
    alist_in_atlas = []
    nlist_in_atlas = []
    mlist_in_atlas = []
    for sno, s in enumerate(tqdm(subjects)):
        mseg_onehot = np.eye(nlabels_small)[mri_man_segs_recoded[sno].data]
        norm = (mri_norms[sno].data / mri_norms[sno].data.max())[np.newaxis, ..., np.newaxis]
        pred = model.predict(norm)
        # transform = gens.vxm_model.predict([l, norm, norm_atlas])
        vxm_model = gens.read_vxm_model(inshape)
        # smooth_wt_input = KL.Input(vxm_model.inputs[0].shape[1:])
        # print("vxm_model.inputs[0]",vxm_model.inputs[0],smooth_wt_input )
        l = np.zeros((1, 1))
        l[0,0] = .3
        #transform = vxm_model.predict([l, norm, norm_atlas])
        transform = vxm_model.predict([l, norm, norm_atlas])
        # transform = transform[:, psize:-psize, psize:-psize, psize:-psize, :]
        dice = lfunc(tf.convert_to_tensor(mseg_onehot[np.newaxis], tf.float32), 
                     tf.convert_to_tensor(pred, tf.float32))
        dice_list.append(dice.numpy())
        ev = (mseg_onehot - pred[0])
        evol = (ev**2).sum(axis=-1)
        elist.append(evol)
        evol_in_atlas = vxm.layers.SpatialTransformer(interp_method='linear', fill_value=0)([evol[np.newaxis, ..., np.newaxis], transform])
        norm_in_atlas = vxm.layers.SpatialTransformer(interp_method='linear', fill_value=0)([norm, transform])
        aseg_in_atlas = vxm.layers.SpatialTransformer(interp_method='linear', fill_value=0)([pred[0][np.newaxis, ...], transform])
        mseg_in_atlas = vxm.layers.SpatialTransformer(interp_method='linear', fill_value=0)([mseg_onehot[np.newaxis, ...], transform])
        elist_in_atlas.append(evol_in_atlas.numpy().squeeze())
        nlist_in_atlas.append(norm_in_atlas.numpy().squeeze())
        #alist_in_atlas.append(np.argmax(aseg_in_atlas.numpy(), axis=-1).squeeze())
        #mlist_in_atlas.append(np.argmax(mseg_in_atlas.numpy(), axis=-1).squeeze())
        alist_in_atlas.append(aseg_in_atlas.numpy().squeeze())
        mlist_in_atlas.append(mseg_in_atlas.numpy().squeeze())

    evol_avg = np.array(elist_in_atlas).mean(axis=0)
    nvol_avg = np.array(nlist_in_atlas).mean(axis=0)
    nbhd_img = scipy.ndimage.convolve(evol_avg, np.ones((patch_size,)*3)/(patch_size**3), mode='constant')
    mseg_avg = np.argmax(np.array(mlist_in_atlas).mean(axis=0), axis=-1)
    aseg_avg = np.argmax(np.array(alist_in_atlas).mean(axis=0), axis=-1)
    max_ind = np.argmax(nbhd_img)
    print('computing index occurence volumes')
    aseg_inds = np.argmax(np.array(alist_in_atlas), axis=-1)
    mseg_inds = np.argmax(np.array(mlist_in_atlas), axis=-1)
    max_inds = 10
    mseg_ind_vol = np.zeros(target_shape + (max_inds,))
    aseg_ind_vol = np.zeros(target_shape + (max_inds,))
    mseg_num_inds = np.zeros(target_shape)
    aseg_num_inds = np.zeros(target_shape)
    for x in tqdm(range(aseg_inds.shape[1])):
        for y in range(aseg_inds.shape[2]):
            for z in range(aseg_inds.shape[3]):
                ind_list = -1 * np.ones((max_inds,))
                u = np.unique(mseg_inds[:, x, y, z])
                for lno, l in enumerate(u):
                    if lno >= max_ind:
                        break
                    mseg_ind_vol[x, y, z, lno] = l
                
                mseg_num_inds[x, y, z] = len(u)
                ind_list = -1 * np.ones((max_inds,))
                u = np.unique(aseg_inds[:, x, y, z])
                for lno, l in enumerate(u):
                    aseg_ind_vol[x, y, z, lno] = l

                aseg_num_inds[x, y, z] = len(u)

    subs = np.unravel_index(max_ind, nbhd_img.shape)
    labels_mseg = np.unique(np.argmax(np.array(mlist_in_atlas), axis=-1)[:, subs[0], subs[1], subs[2]])
    labels_aseg = np.unique(np.argmax(np.array(alist_in_atlas), axis=-1)[:, subs[0], subs[1], subs[2]])
    print(subs)

# fv = fs.Fre
    # fv = fs.Freeview()
    # fv.vol(mri_segs[0].copy(nvol_avg), name='norm avg')
    # fv.vol(mri_segs[0].copy(mseg_avg), name='mseg', opts=':colormap=lut:lut=nonlat')
    # fv.vol(mri_segs[0].copy(aseg_avg), name='aseg', opts=':colormap=lut:lut=nonlat')
    # nvols = 3
    # mvols = np.argmax(np.array(mlist_in_atlas[0:nvols]), axis=-1)
    # avols = np.argmax(np.array(alist_in_atlas[0:nvols]), axis=-1)
    # fv.vol(mri_segs[0].copy(np.transpose(np.array(nlist_in_atlas[0:nvols]), (1, 2, 3, 0))), name='norm vols')
    # fv.vol(mri_segs[0].copy(np.transpose(mvols, (1, 2, 3, 0))), name='man segs', opts=':colormap=lut:lut=nonlat')
    # fv.vol(mri_segs[0].copy(np.transpose(avols, (1, 2, 3, 0))), name='synth segs', opts=':colormap=lut:lut=nonlat')
    # fv.vol(mri_segs[0].copy(nbhd_img), name='err avg', opts=':colormap=heat:heatscale=.25,.5')
    # fv.show(opts=f'-slice {subs[0]} {subs[1]} {subs[2]}', verbose=True)


reading aseg atlas model from aseg.fscale.1.1.h5


  0%|          | 0/41 [00:00<?, ?it/s]

1/1 [==============================] - 155s 155s/step


  2%|▏         | 1/41 [05:03<3:22:24, 303.60s/it]

1/1 [==============================] - 164s 164s/step


...

1/1 [==============================] - 170s 170s/step


 22%|██▏       | 9/41 [47:05<2:48:55, 316.72s/it]

1/1 [==============================] - 164s 164s/step


 24%|██▍       | 10/41 [52:18<2:43:05, 315.65s/it]

1/1 [==============================] - 113s 113s/step


In [16]:
mri_hard_seg = sf.Volume(hard_seg).resample_like(mri_seg_atlas).reshape(target_shape)



In [ ]:
print(imgs[0].shape)
nvols = 3
mvols = np.argmax(np.array(mlist_in_atlas[0:nvols]), axis=-1)
avols = np.argmax(np.array(alist_in_atlas[0:nvols]), axis=-1)
volume_data = mri_segs[0].copy(np.transpose(np.array(nlist_in_atlas[0:nvols]), (1, 2, 3, 0))).astype(np.int32)
nib.save(nib.Nifti1Image(volume_data, np.eye(4), header=None), "output/nvol_avg.nii.gz")

# nib.save(nib.Nifti1Image(mri_segs[0].copy(np.transpose(np.array(nlist_in_atlas[0:nvols]), (1, 2, 3, 0))).astype(np.int32), np.eye(4), header=None), "output/nvol_avg.nii.gz")

# nib.save(nib.Nifti1Image(tseg[i].astype(np.int32), np.eye(4), header=None), f"output/tseg{i}.nii.gz")
# nib.save(nib.Nifti1Image(pseg[i].astype(np.int32), np.eye(4), header=None), f"output/pseg{i}.nii.gz")
    # for i in range(imgs.shape[0]):
        # nib.save(nib.Nifti1Image(imgs[i].astype(np.int32), np.eye(4), header=None), f"output/nvol_avg.nii.gz")
        # nib.save(nib.Nifti1Image(tseg[i].astype(np.int32), np.eye(4), header=None), f"output/tseg{i}.nii.gz")
        # nib.save(nib.Nifti1Image(pseg[i].astype(np.int32), np.eye(4), header=None), f"output/pseg{i}.nii.gz")
